In [5]:
import cv2
import numpy as np
import os

def grayscale(image):
    """Convert the image to grayscale."""
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def gaussian_blur(image, kernel_size=5):
    """Apply Gaussian blur to the image."""
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

def canny(image, low_threshold=50, high_threshold=150):
    """Apply Canny edge detection to the image."""
    return cv2.Canny(image, low_threshold, high_threshold)

def region_of_interest(image):
    """Apply an image mask to keep the region of the image defined by the polygon."""
    height, width = image.shape[:2]
    mask = np.zeros_like(image)

     # Define a polygon with the vertices based on the image dimensions
    Height_updated = 500
    weidth_updated = 700
    polygon = np.array([[
        (0, Height_updated),
        (weidth_updated, Height_updated),
        (0, int(0.7 * Height_updated)),
        (weidth_updated, int(0.7 * Height_updated))
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)

    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def hough_lines(image):
    """Detect lines in the image using Hough Transform."""
    return cv2.HoughLinesP(image, rho=1, theta=np.pi/180, threshold=50, minLineLength=50, maxLineGap=200)

def draw_lines(image, lines, min_slope=0.5):
    """Draw lines on the image and filter out horizontal lines."""
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                # Calculate the slope of the line
                if x2 - x1 == 0:  # Avoid division by zero
                    slope = float('inf')
                else:
                    slope = (y2 - y1) / (x2 - x1)
                
                # Filter out horizontal lines
                if abs(slope) < min_slope:
                    continue
                
                cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

def weighted_img(line_image, initial_image, α=0.8, β=1., λ=0.):
    """Combine the initial image with the line image."""
    return cv2.addWeighted(initial_image, α, line_image, β, λ)

def process_frame(frame):
    """Process a single frame to detect lanes."""
    gray = grayscale(frame)
    blur = gaussian_blur(gray)
    edges = canny(blur)
    roi = region_of_interest(edges)
    lines = hough_lines(roi)
    line_image = draw_lines(frame, lines)
    result = weighted_img(line_image, frame)
    return result

# Path to the input video
input_video_path = "C:/Users/gaurav/Videos/input_video.mp4"
output_video_path = 'C:/Users/gaurav/Videos/output_video.mp4'

# Create a VideoCapture object
cap = cv2.VideoCapture(input_video_path)

# Get the width, height, and frames per second (fps) of the video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use 'XVID' or 'X264' as well
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame
    processed_frame = process_frame(frame)

    # Write the processed frame to the output video
    out.write(processed_frame)

# Release everything if job is finished
cap.release()
out.release()

In [2]:
pip install opencv-python-headless


Note: you may need to restart the kernel to use updated packages.
